In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
import eda

%matplotlib inline

/Users/macuser/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/Users/macuser/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### Build Model

In [2]:
df = pd.read_csv('train.csv',index_col='ID')

In [3]:
df.head()

,target,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
ID,,,,,,,,,,,,,,,,,,,,,
3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [4]:
# 21 perfectly correlated w/ variable in v3
df = df.drop('v21', axis=1)
# v91 perfectly correlated with v107 (check this later)
df = df.drop('v91', axis=1)

# Pick best values from large dummies
best_vals = df['v22'].value_counts()[:10].index.values

for val in best_vals:
    df[val] = 1 * (df['v22'] == val) 

df = df.drop('v22', axis=1)    

In [5]:
# df, na_cols = eda.not_applicable(df)
# print 'NA Columns:', na_cols

In [6]:
df['v3'] = df['v3'].fillna('Other')

In [7]:
df_new, str_na_fill = eda.na_fill_str(df)
print 'String NA Fill Columns:', str_na_fill

v30 Other 60110 na rows @ 52.6 % nan's
v31 Other 3457 na rows @ 3.0 % nan's
v52 Other 3 na rows @ 0.0 % nan's
v56 Other 6882 na rows @ 6.0 % nan's
v107 Other 3 na rows @ 0.0 % nan's
v112 Other 382 na rows @ 0.3 % nan's
v113 Other 55304 na rows @ 48.4 % nan's
v125 Other 77 na rows @ 0.1 % nan's
String NA Fill Columns: ['v30', 'v31', 'v52', 'v56', 'v107', 'v112', 'v113', 'v125']


In [8]:
df_new, int_na_fill = eda.na_fill_int(df_new)
print 'Value NA Fill Columns:', int_na_fill

v1 meaned 49832 na rows @ 43.6 % nan's
v2 meaned 49796 na rows @ 43.6 % nan's
v4 meaned 49796 na rows @ 43.6 % nan's
v5 meaned 48624 na rows @ 42.5 % nan's
v6 meaned 49832 na rows @ 43.6 % nan's
v7 meaned 49832 na rows @ 43.6 % nan's
v8 meaned 48619 na rows @ 42.5 % nan's
v9 meaned 49851 na rows @ 43.6 % nan's
v10 meaned 84 na rows @ 0.1 % nan's
v11 meaned 49836 na rows @ 43.6 % nan's
v12 meaned 86 na rows @ 0.1 % nan's
v13 meaned 49832 na rows @ 43.6 % nan's
v14 meaned 4 na rows @ 0.0 % nan's
v15 meaned 49836 na rows @ 43.6 % nan's
v16 meaned 49895 na rows @ 43.6 % nan's
v17 meaned 49796 na rows @ 43.6 % nan's
v18 meaned 49832 na rows @ 43.6 % nan's
v19 meaned 49843 na rows @ 43.6 % nan's
v20 meaned 49840 na rows @ 43.6 % nan's
v23 meaned 50675 na rows @ 44.3 % nan's
v25 meaned 48619 na rows @ 42.5 % nan's
v26 meaned 49832 na rows @ 43.6 % nan's
v27 meaned 49832 na rows @ 43.6 % nan's
v28 meaned 49832 na rows @ 43.6 % nan's
v29 meaned 49832 na rows @ 43.6 % nan's
v32 meaned 49832 na r

In [9]:
# CREATE Dummy Columns and insert Boolean values
df_newer, dummies, bools = eda.dummy(df_new)

print 'Dummy Columns:', dummies
for i in dummies:
    print "Dummy", i, df_new[i].unique()
    
print ""    
    
print 'Boolean Columns:', bools
for i in bools:
    print "Boolean", i, df_new[i].unique()

v3
v24
v30
v31
v38
v47
v52
v56
v62
v66
v71
v72
v74
v75
v79
v107
v110
v112
v113
v125
v129
Dummy Columns: ['v3', 'v24', 'v30', 'v31', 'v38', 'v47', 'v52', 'v56', 'v62', 'v66', 'v71', 'v72', 'v74', 'v75', 'v79', 'v107', 'v110', 'v112', 'v113', 'v125', 'v129']
Dummy v3 ['Cv3' 'Otherv3' 'Av3' 'Bv3']
Dummy v24 ['C' 'E' 'D' 'A' 'B']
Dummy v30 ['Cv30' 'Otherv30' 'Gv30' 'Dv30' 'Bv30' 'Fv30' 'Av30' 'Ev30']
Dummy v31 ['Av31' 'Bv31' 'Otherv31' 'Cv31']
Dummy v38 [ 0  4  1  2  6  5  8  3 10  9 12  7]
Dummy v47 ['C' 'E' 'I' 'D' 'F' 'G' 'J' 'B' 'A' 'H']
Dummy v52 ['Gv52' 'Fv52' 'Hv52' 'Kv52' 'Av52' 'Cv52' 'Ev52' 'Jv52' 'Dv52' 'Iv52'
 'Lv52' 'Bv52' 'Otherv52']
Dummy v56 ['DIv56' 'DYv56' 'ASv56' 'BWv56' 'Otherv56' 'DXv56' 'DPv56' 'AFv56' 'CYv56'
 'AWv56' 'ALv56' 'AZv56' 'Zv56' 'DNv56' 'BQv56' 'BMv56' 'AGv56' 'BJv56'
 'BVv56' 'Rv56' 'BLv56' 'CNv56' 'Uv56' 'Nv56' 'Vv56' 'BUv56' 'Pv56' 'DOv56'
 'BZv56' 'DLv56' 'DSv56' 'DFv56' 'DHv56' 'DJv56' 'BAv56' 'BSv56' 'BXv56'
 'DZv56' 'BKv56' 'COv56' 'CIv56' 'Gv56' '

In [10]:
df_newer['target'] = 1 * (df_newer['target'] == 1)
df_newer.head()

,target,v1,v2,v4,v5,v6,v7,v8,v9,v10,...,Zv125v125,0v129,11v129,1v129,2v129,3v129,5v129,6v129,7v129,8v129
ID,,,,,,,,,,,,,,,,,,,,,
3,1,1.335739,8.727474,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1.630686,7.464411,4.145098,9.191265,2.436402,2.483921,2.301630,9.031859,1.312910,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,0.943877,5.310079,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,1,0.797415,8.304757,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,1.630686,7.464411,4.145098,8.742359,2.436402,2.483921,1.496569,9.031859,1.050328,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Function to drop highly correlated columns from features

In [11]:
def corr_df(df, corr_val):
    
    corr_matrix = df.corr()
    iters = xrange(len(corr_matrix.columns) - 1)
    drop_cols = []
    
    for i in iters:
        for j in xrange(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = item.values
            if val > corr_val:
                print col.values[0], "|", row.values[0], "|", round(val[0][0],2)
                drop_cols.append(i)
    
    drops = sorted(set(drop_cols))[::-1]
#     print drops
    print len(drops)
    
    for i in drops:
        col = df.iloc[:, (i+1):(i+2)].columns.values
        df = df.drop(col, axis=1)
    
    return df, drops

In [ ]:
print "co1 1 | col 2 | corr"
df_new_new = corr_df(df_newer, 0.9)

In [ ]:
x = df_new_new.drop(['target'], axis=1)
x = sm.add_constant(x)
y = df_newer.target

In [ ]:
model = LogisticRegression()
fit = model.fit(x, y)

In [ ]:
fit.predict_proba(x)[:,1]

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.01)

In [ ]:
lasso_fit = lasso.fit(x, y)

In [ ]:
probas = lasso_fit.predict(x)
ids = df_newer.index.values

In [ ]:
results = pd.DataFrame(np.vstack((ids, probas)).T)
results.head()

#### Accuracy Rating

In [ ]:
len(x)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df_newer.target,np.around(probas))

### Transform Test Data

In [91]:
test = pd.read_csv('test.csv', index_col='ID')

In [92]:
# 21 perfectly correlated w/ variable in v3
test = test.drop('v21', axis=1)
# v91 perfectly correlated with v107 (check this later)
test = test.drop('v91', axis=1)

# Pick best values from large dummies
best_vals = test['v22'].value_counts()[:10].index.values

for val in best_vals:
    test[val] = 1 * (test['v22'] == val) 

test = test.drop('v22', axis=1)    

In [93]:
test['v3'] = test['v3'].fillna('Other')

In [94]:
test_new, str_na_fill = eda.na_fill_str(test)
print 'String NA Fill Columns:', str_na_fill

v30 Other 60342 na rows @ 93.5 % nan's
v31 Other 3492 na rows @ 5.4 % nan's
v52 Other 2 na rows @ 0.0 % nan's
v56 Other 6918 na rows @ 10.7 % nan's
v107 Other 2 na rows @ 0.0 % nan's
v112 Other 413 na rows @ 0.6 % nan's
v113 Other 55358 na rows @ 85.8 % nan's
v125 Other 84 na rows @ 0.1 % nan's
String NA Fill Columns: ['v30', 'v31', 'v52', 'v56', 'v107', 'v112', 'v113', 'v125']


In [95]:
test_new, int_na_fill = eda.na_fill_int(df_new)
print 'Value NA Fill Columns:', int_na_fill

Value NA Fill Columns: []


In [96]:
# CREATE Dummy Columns and insert Boolean values
test_newer, dummies, bools = eda.dummy(test_new)

print 'Dummy Columns:', dummies
for i in dummies:
    print "Dummy", i, test_new[i].unique()
    
print ""    
    
print 'Boolean Columns:', bools
for i in bools:
    print "Boolean", i, test_new[i].unique()

v3
v24
v30
v31
v38
v47
v52
v56
v62
v66
v71
v72
v74
v75
v79
v107
v110
v112
v113
v125
v129
Dummy Columns: ['v3', 'v24', 'v30', 'v31', 'v38', 'v47', 'v52', 'v56', 'v62', 'v66', 'v71', 'v72', 'v74', 'v75', 'v79', 'v107', 'v110', 'v112', 'v113', 'v125', 'v129']
Dummy v3 ['Cv3v3' 'Otherv3v3' 'Bv3v3' 'Av3v3']
Dummy v24 ['E' 'D' 'B' 'C' 'A']
Dummy v30 ['Otherv30' 'Av30' 'Cv30' 'Ev30' 'Dv30' 'Gv30' 'Fv30' 'Bv30']
Dummy v31 ['Av31' 'Bv31' 'Otherv31' 'Cv31']
Dummy v38 [ 0  4  1  3  2  5  6  9  8  7 10 12]
Dummy v47 ['J' 'C' 'I' 'G' 'F' 'D' 'E' 'A' 'B']
Dummy v52 ['Cv52' 'Av52' 'Hv52' 'Dv52' 'Lv52' 'Gv52' 'Bv52' 'Iv52' 'Jv52' 'Fv52'
 'Ev52' 'Kv52' 'Otherv52']
Dummy v56 ['AFv56' 'DIv56' 'DOv56' 'CNv56' 'Otherv56' 'DPv56' 'BJv56' 'BWv56' 'Uv56'
 'ASv56' 'BLv56' 'DJv56' 'CYv56' 'BKv56' 'CPv56' 'BZv56' 'Pv56' 'CSv56'
 'DFv56' 'AWv56' 'AGv56' 'BMv56' 'DHv56' 'Vv56' 'Gv56' 'DSv56' 'DYv56'
 'BVv56' 'BXv56' 'CIv56' 'BYv56' 'Nv56' 'Av56' 'DXv56' 'ANv56' 'DAv56'
 'ALv56' 'Yv56' 'AOv56' 'Zv56' 'AZv56' 'ARv56

In [122]:
print "co1 1 | col 2 | corr"
test_new_new = corr_df(test_newer, 0.85)

co1 1 | col 2 | corr
v12 | v10 | 0.91
v25 | v8 | 0.94
v32 | v15 | 0.91
v37 | v1 | 0.89
v41 | v29 | 0.9
v43 | v26 | 0.91
v46 | v8 | 0.98
v46 | v25 | 0.98
v49 | v41 | 0.9
v53 | v11 | 0.96
v54 | v8 | 0.85
v54 | v25 | 0.97
v54 | v46 | 0.92
v55 | v33 | 0.94
v60 | v26 | 0.98
v60 | v43 | 0.88
v63 | v8 | 0.98
v63 | v25 | 0.98
v63 | v46 | 0.99
v63 | v54 | 0.92
v64 | v17 | 0.98
v64 | v44 | 0.86
v65 | v20 | 0.91
v67 | v29 | 0.92
v67 | v41 | 0.93
v73 | v15 | 0.95
v73 | v32 | 0.91
v76 | v17 | 0.99
v76 | v64 | 0.95
v77 | v29 | 0.92
v77 | v41 | 0.87
v77 | v67 | 0.91
v83 | v33 | 0.97
v83 | v55 | 0.9
v86 | v15 | 0.86
v86 | v32 | 0.91
v89 | v8 | 0.87
v89 | v25 | 0.96
v89 | v46 | 0.93
v89 | v54 | 0.95
v89 | v63 | 0.93
v93 | v17 | 0.89
v93 | v64 | 0.88
v93 | v76 | 0.87
v95 | v92 | 0.98
v96 | v29 | 0.96
v96 | v41 | 0.95
v96 | v67 | 0.87
v96 | v77 | 0.88
v103 | v94 | 0.86
v104 | v13 | 0.93
v105 | v8 | 0.92
v105 | v25 | 0.96
v105 | v46 | 0.95
v105 | v54 | 0.94
v105 | v63 | 0.94
v105 | v89 | 0.92
v106 | v17 |

In [134]:
print x_test.columns
print x.columns

Index([u'const', u'v1', u'v2', u'v4', u'v5', u'v6', u'v7', u'v8', u'v9',
       u'v10',
       ...
       u'Zv125v125', u'0v129', u'11v129', u'1v129', u'2v129', u'3v129',
       u'5v129', u'6v129', u'7v129', u'8v129'],
      dtype='object', length=448)
Index([u'const', u'v1', u'v2', u'v4', u'v5', u'v6', u'v7', u'v8', u'v9',
       u'v10',
       ...
       u'Zv125v125', u'0v129', u'11v129', u'1v129', u'2v129', u'3v129',
       u'5v129', u'6v129', u'7v129', u'8v129'],
      dtype='object', length=456)


In [137]:
for i, item in enumerate(x.columns):
    if item != x_test.columns[i]:
        print item, x_test.columns[i]

QKI PWR
PWR QKI
GBS VZF
Av3 Bv3v3
Bv3 Cv3v3
Cv3 Otherv3v3
Otherv31v31 0v38
0v38 10v38
10v38 12v38
12v38 1v38
1v38 2v38
2v38 3v38
3v38 4v38
4v38 5v38
5v38 6v38
6v38 8v38
8v38 9v38
9v38 Av47
Av47 Bv47
Bv47 Cv47
Cv47 Ev47
Ev47 Fv47
Fv47 Iv47
Hv47 Jv47
Iv47 Av52v52
Jv47 Bv52v52
Av52v52 Cv52v52
Bv52v52 Dv52v52
Cv52v52 Ev52v52
Dv52v52 Fv52v52
Ev52v52 Gv52v52
Fv52v52 Hv52v52
Gv52v52 Jv52v52
Hv52v52 Kv52v52
Jv52v52 Lv52v52
Kv52v52 Otherv52v52
Lv52v52 AAv56v56
Otherv52v52 ABv56v56
AAv56v56 ACv56v56
ABv56v56 ADv56v56
ACv56v56 AEv56v56
AEv56v56 AFv56v56
AFv56v56 AGv56v56
AGv56v56 AHv56v56
AHv56v56 AIv56v56
AIv56v56 AJv56v56
AJv56v56 AKv56v56
AKv56v56 ALv56v56
ALv56v56 AMv56v56
AMv56v56 ANv56v56
ANv56v56 AOv56v56
AOv56v56 AQv56v56
APv56v56 ARv56v56
ARv56v56 ASv56v56
ASv56v56 AUv56v56
ATv56v56 AVv56v56
AUv56v56 AWv56v56
AVv56v56 AYv56v56
AWv56v56 AZv56v56
AYv56v56 Av56v56
AZv56v56 BAv56v56
Av56v56 BBv56v56
BAv56v56 BCv56v56
BCv56v56 BDv56v56
BDv56v56 BEv56v56
BEv56v56 BFv56v56
BFv56v56 BGv56v56
BGv

IndexError: index 448 is out of bounds for axis 0 with size 448

x_test = test_new_new
x_test = sm.add_constant(x_test)
test_probas = lasso_fit.predict(x_test)
test_ids = df_newer.index.values

In [ ]:
test_results = pd.DataFrame(np.vstack((test_ids, test_probas)).T)
test_results.head()